In [86]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import feedparser
import os
import numpy as np

In [25]:
#Use https://github.com/sangaline/wayback-machine-scraper to scrape all the available snapshots for the rss feeds

#wayback-machine-scraper -a 'http://feeds.bbci.co.uk/afrique/rss.xml$' http://feeds.bbci.co.uk/afrique/rss.xml
#wayback-machine-scraper -a 'http://feeds.bbci.co.uk/afrique/monde/rss.xml$' http://feeds.bbci.co.uk/afrique/monde/rss.xml
#wayback-machine-scraper -a 'http://feeds.bbci.co.uk/afrique/region/rss.xml$' http://feeds.bbci.co.uk/afrique/monde/rss.xml
#wayback-machine-scraper -a 'http://feeds.bbci.co.uk/afrique/sports/rss.xml$' http://feeds.bbci.co.uk/afrique/sports/rss.xml


In [26]:
import os

# Getting the current work directory (cwd)
thisdir = os.getcwd()
snapshots = []

# r=root, d=directories, f = files
for r, d, f in os.walk(thisdir):
    for file in f:
        if file.endswith(".snapshot"):
            snapshots.append(os.path.join(r, file))

In [71]:
data = pd.DataFrame([])
for snap in snapshots:
    parser = feedparser.parse(snap)
    for post in parser.entries:
        try:
            title = post.title
            link = post.link
            date = post.published
            data = data.append(pd.DataFrame({'title': title, 'url': link,'date' : date}, index=[0]), ignore_index=True)
        except:
            continue
    

In [72]:
data.shape

(2541, 3)

In [75]:
def categorize(x):
    if 'region' in x:
        return "afrique"
    elif 'sports' in x:
        return "sport"
    elif 'monde' in x:
        return "monde"
    else:
        return np.nan


In [77]:
data['category'] = data["url"].apply (lambda x: categorize(x))

In [82]:
data.head()

,title,url,date,category
0,"Nigeria, le président du Sénat convoqué pour a...",http://www.bbc.com/afrique/44364579,"Mon, 04 Jun 2018 21:54:22 GMT",NaN
1,Un nouveau premier ministre pour Madagascar,http://www.bbc.com/afrique/region-44356472,"Mon, 04 Jun 2018 13:00:45 GMT",afrique
2,"Au Guatemala, 25 morts dans l'éruption d'un vo...",http://www.bbc.com/afrique/monde-44324063,"Mon, 04 Jun 2018 12:15:25 GMT",monde
3,Alassane Ouattara et son 3ème mandat,http://www.bbc.com/afrique/region-44356470,"Mon, 04 Jun 2018 10:22:56 GMT",afrique
4,13 personnes tuées dans des attaques au Nigeria,http://www.bbc.com/afrique/region-44351196,"Mon, 04 Jun 2018 03:07:49 GMT",afrique


In [83]:
data = data.drop_duplicates(subset=['url', 'title'], keep='last')

In [85]:
data.shape

(2121, 4)

In [87]:
def extract_userneed(x):
    page=requests.get(x)
    soup=BeautifulSoup(page.text,'html.parser')
    for i in soup.findAll("script"):
        try:
            return re.findall(r'(?<="campaignName".").+?(?=\")', str(i.string))[0]
        except:
            continue

In [99]:
def get_text(url):
    try:
        article = requests.get(url)
        soup = BeautifulSoup(article.content, 'html.parser')
        body = soup.find(role='main')
        text = [p.text for p in body.find_all('p',dir="ltr")]
        return ''.join(text)
    except:
        return "error"

In [89]:
%%time
data["User Need"] = data["url"].apply(lambda x : extract_userneed(x))

CPU times: user 2min 59s, sys: 6.54 s, total: 3min 5s
Wall time: 23min 5s


In [94]:
data["User Need"] = data["User Need"].apply(lambda s: re.sub(r"(WS - )", "", s) if pd.notnull(s) else s)


In [95]:
data["User Need"].value_counts()

Update me              1106
Keep me on trend        479
Educate me              136
Give me perspective     129
Inspire me              114
CS                       60
Divert me                28
News - Learning           1
Name: User Need, dtype: int64

In [100]:
%%time
data["text"] = data["url"].apply(lambda x : get_text(x))

CPU times: user 2min 25s, sys: 4.59 s, total: 2min 29s
Wall time: 6min 54s


In [104]:
data[data.text=="error"]

,title,url,date,category,User Need,text
790,Enterrés dans des tombes sans nom,http://www.bbc.co.uk/news/resources/idt-54e83d...,"Thu, 12 May 2016 10:40:19 GMT",NaN,None,error
1210,Guide de rédaction,http://www.bbc.co.uk/academy/french/language/s...,"Tue, 26 Jan 2016 13:45:50 GMT",NaN,None,error


In [106]:
data = data.dropna(subset=["User Need"])

In [109]:
data.to_csv("bbc_afrique_rss.csv",index = False)

In [111]:
data2 = pd.read_csv("bbc_afrique_userneeds.csv")

In [114]:
data3 = data2.append(data)

In [115]:
data3 = data3.drop_duplicates(subset=['url'], keep='last')

In [119]:
data3.to_csv("BBC_afrique_final.csv",index=False)

In [122]:
data3["User Need"].value_counts()

Update me                1448
Keep me on trend          821
Educate me                706
Give me perspective       627
Inspire me                441
CS                        245
Divert me                  86
News - Disclosure          14
News - Learning             5
News - Reader Revenue       1
Name: User Need, dtype: int64